In [ ]:
#database demo là dữ liệu nguồn
#database demo1 là data warehouse

In [10]:
import psycopg2

conn_demo = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)

cursor_demo = conn_demo.cursor()

conn_demo1 = psycopg2.connect(
    host="localhost",
    database="demo1",
    user="postgres",
    password="password"
)

cursor_demo1 = conn_demo1.cursor()

cursor_demo.execute("SELECT id, fullname, phone, created_at FROM tbl_user")
users = cursor_demo.fetchall()

for user in users:
    cursor_demo1.execute("""
        INSERT INTO user_dim (id, fullname, phone, created_at)
        VALUES (%s, %s, %s, %s)
    """, (user[0], user[1], user[2], user[3]))

cursor_demo.execute("SELECT id, price, image, created_at FROM tbl_product")
products = cursor_demo.fetchall()

for product in products:
    cursor_demo1.execute("""
        INSERT INTO product_dim (id, price, image, created_at)
        VALUES (%s, %s, %s, %s)
    """, (product[0], product[1], product[2], product[3]))

cursor_demo.execute("select id, order_id, product_id, price, quantity, created_at FROM tbl_order_item")
orders = cursor_demo.fetchall()

for order in orders:
    cursor_demo1.execute("""
        INSERT INTO order_item_dim (id, order_id, product_id, price, quantity, created_at)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (order[0], order[1], order[2], order[3], order[4], order[5]))


conn_demo1.commit()
cursor_demo1.close()
conn_demo1.close()

In [29]:
import psycopg2
from psycopg2 import sql
from datetime import datetime

conn_demo = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor_demo = conn_demo.cursor()

conn_demo1 = psycopg2.connect(
    dbname="demo1",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"
)
cursor_demo1 = conn_demo1.cursor()

select_query = """
    SELECT id, created_at
    FROM tbl_order
"""
cursor_demo.execute(select_query)
orders = cursor_demo.fetchall()

for order in orders:
    order_id = order[0]
    created_at = order[1]
    
    year = created_at.year
    quarter = (created_at.month - 1) // 3 + 1
    month = created_at.month

    created_at_str = created_at.strftime('%Y-%m-%d %H:%M:%S')

    insert_query = sql.SQL("""
        INSERT INTO time_dim (id, day, week, month, quarter, year, created_at)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """).format(
        sql.Identifier(order_id),
        sql.Identifier(str(created_at.day)),
        sql.Identifier(str(0)),
        sql.Identifier(str(month)),
        sql.Identifier(str(quarter)),
        sql.Identifier(str(year)),
        sql.Identifier(created_at_str),
    )

    cursor_demo1.execute(insert_query, (order_id, created_at.day, 0, month, quarter, year, created_at_str))

conn_demo1.commit()
cursor_demo.close()
cursor_demo1.close()
conn_demo.close()
conn_demo1.close()



In [ ]:
import psycopg2

source_conn = psycopg2.connect(
    dbname="demo",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"


source_cur = source_conn.cursor()

target_conn = psycopg2.connect(
    dbname="demo1",
    user="postgres",
    password="password",
    host="localhost",
    port="5432"


target_cur = target_conn.cursor()

source_cur.execute("SELECT * FROM tbl_order")
rows = source_cur.fetchall()

for row in rows:
    id, user_id, payment_type, discount, total_cost, fee_total, created_at = row
    target_cur.execute("INSERT INTO order_dim (id, user_id, payment_type, discount, total_cost, fee_total, created_at) "
                       "VALUES (%s, %s, %s, %s, %s, %s, %s)",
                       (id, user_id, payment_type, discount, total_cost, fee_total, created_at))

target_conn.commit()

source_cur.close()
source_conn.close()
target_cur.close()
target_conn.close()